In [19]:
import os 
import sys
import pandas as pd
import numpy as np

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.chdir('../')

In [10]:
from src.dataset import load_dataset

Cargamos los datos

In [11]:
X_train, X_test, y_train, y_test = load_dataset(
        filename = 'dataset_SCL.csv',
        synthetic_filename='synthetic_features.csv',
        seed=43,
        val_size=0.2
    )

   Temp-A  Per-D  ATRASADO
0    True  night     False
1    True  night     False
2    True  night     False
3    True  night     False
4    True  night      True
  MES     DIANOM TIPOVUELO              OPERA  SIGLAORI SIGLADES Temp-A  Per-D
0   1    Domingo         I  American Airlines  Santiago    Miami   True  night
1   1      Lunes         I  American Airlines  Santiago    Miami   True  night
2   1     Martes         I  American Airlines  Santiago    Miami   True  night
3   1  Miercoles         I  American Airlines  Santiago    Miami   True  night
4   1     Jueves         I  American Airlines  Santiago    Miami   True  night
---------- Dataset Info ----------

Number of datapoints: 68206
Number of datapoints train: 54564
Percentage positive examples train: 0.45007697382889816
Number of datapoints test: 13642
Percentage positive examples test: 0.4489810878170356
Number of features per row: 8


Queremos sin split asique se vuelven a juntar

In [20]:
y_train

array([1, 1, 1, ..., 1, 1, 0])

In [21]:
X = pd.concat([X_train, X_test], axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [23]:
from xgboost import XGBClassifier

Establecemos los parametros y valores a probar

In [70]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0, 0.1, 0.2, 0.5, 1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5, 6, 7, 8],
        'learning_rate': [0.1, 0.15, 0.2, 0.3],
        'n_estimators': [700, 800, 900, 1000]
        }

Instanciamos el modelo

In [54]:
xgb = XGBClassifier(tree_method="hist",
                    silent=True, nthread=1, enable_categorical=True,)

In [26]:
import time
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

Utilizamos la busqueda de hiperparamteros, utilizando 3-folds estratificados (presencia de clases) para evaluar el resultado segun la metrica AUC

In [66]:
folds = 3

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=10, scoring='roc_auc', n_jobs=-1, cv=skf.split(X,y), verbose=3, random_state=1001 )

# Here we go
start = time.time()
random_search.fit(X, y)
end = time.time()
print(end - start)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1685694848790/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[22:07:08] WARNING: /Users/runner/miniforge3/

In [67]:
print(f"Best tree {random_search.best_estimator_}")

Best tree XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=None, nthread=1, num_parallel_tree=None,
              predictor=None, ...)


Obtenemos el mejor resultado

In [68]:
print(f"Best score {random_search.best_score_}")

Best score 0.6288601123480311


Guardamos los hiperparametros

In [69]:
print(random_search.best_params_)

{'subsample': 1.0, 'n_estimators': 700, 'min_child_weight': 10, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.6}
